# Add, replace, stage and delete artifacts

In [ ]:
instance_name = "lamindb-ci-test-add-replace-stage"

!lamin init --storage s3://lamindb-ci --name {instance_name}

In [ ]:
import pytest
import lamindb as ln
from lamindb_setup.dev.upath import UPath
from lamindb.dev.storage.file import AUTO_KEY_PREFIX

## Save with auto-managed (`key=None`)

In [ ]:
root = ln.settings.storage

In [ ]:
artifact = ln.Artifact("./test-files/iris.csv", description="iris.csv")

In [ ]:
artifact.save()

In [ ]:
key_path = root / f"{AUTO_KEY_PREFIX}{artifact.uid}{artifact.suffix}"

In [ ]:
assert key_path.exists()

In [ ]:
cache_csv_path = artifact.stage()

In [ ]:
cache_csv_path

In [ ]:
assert cache_csv_path.suffix == ".csv"

In [ ]:
artifact.replace("./test-files/iris.data")

In [ ]:
artifact.save()

In [ ]:
old_key_path = key_path
new_key_path = root / f"{AUTO_KEY_PREFIX}{artifact.uid}{artifact.suffix}"

The suffix changed:

In [ ]:
old_key_path

In [ ]:
new_key_path

In [ ]:
assert not old_key_path.exists()
assert new_key_path.exists()

In [ ]:
cache_data_path = artifact.stage()

In [ ]:
cache_data_path

In [ ]:
assert cache_data_path.suffix == ".data"

In [ ]:
assert cache_data_path.stat().st_mtime >= cache_csv_path.stat().st_mtime

In [ ]:
artifact.delete(permanent=True)

## Save with manually passed real `key`

In [ ]:
ln.settings.artifact_use_virtual_keys = False

In [ ]:
artifact = ln.Artifact("./test-files/iris.csv", key="iris.csv")

In [ ]:
artifact.save()

In [ ]:
key_path = root / "iris.csv"

In [ ]:
assert key_path.exists()

In [ ]:
artifact.replace("./test-files/new_iris.csv")

In [ ]:
artifact.save()

Check paths: no changes here, as the suffix didn't change.

In [ ]:
old_key_path = key_path
new_key_path = root / "new_iris.csv"

In [ ]:
old_key_path

In [ ]:
new_key_path

In [ ]:
assert old_key_path.exists()
assert not new_key_path.exists()

In [ ]:
artifact.replace("./test-files/iris.data")

In [ ]:
artifact.save()

In [ ]:
new_key_path = root / "iris.data"

In [ ]:
old_key_path

In [ ]:
new_key_path

In [ ]:
assert not old_key_path.exists()
assert new_key_path.exists()

In [ ]:
artifact.delete(permanent=True, storage=True)

## Save from memory

In [ ]:
import pandas as pd

In [ ]:
iris = pd.read_csv("./test-files/iris.csv")

In [ ]:
artifact = ln.Artifact(data=iris, description="iris_store", key="iris.parquet")

In [ ]:
artifact.save()

In [ ]:
key_path = root / "iris.parquet"

In [ ]:
assert key_path.exists()

In [ ]:
artifact.replace(data=iris[:-1])

In [ ]:
assert artifact.key == "iris.parquet"

In [ ]:
artifact.save()

In [ ]:
assert key_path.exists()

In [ ]:
artifact.replace("./test-files/new_iris.csv")

In [ ]:
artifact.save()

In [ ]:
old_key_path = key_path
new_key_path = root / "iris.csv"

In [ ]:
old_key_path

In [ ]:
new_key_path

In [ ]:
assert not old_key_path.exists()
assert new_key_path.exists()

In [ ]:
# we use the path in the next section
path_in_storage = artifact.path

artifact.delete(permanent=True, storage=False)

## Save with manually passed virtual `key`

In [ ]:
ln.settings.artifact_use_virtual_keys = True

In [ ]:
artifact = ln.Artifact("./test-files/iris.csv", key="iris.csv")

In [ ]:
artifact.save()

In [ ]:
with pytest.raises(ValueError):
    artifact.replace(path_in_storage)

In [ ]:
# return an existing artifact if the hash is the same
assert artifact == artifact.replace("./test-files/iris.csv")

In [ ]:
fpath = artifact.path
assert fpath.suffix == ".csv" and fpath.stem == artifact.uid

In [ ]:
artifact.replace("./test-files/iris.data")

In [ ]:
artifact.save()

In [ ]:
assert artifact.key == "iris.data"

In [ ]:
assert not fpath.exists()

In [ ]:
fpath = artifact.path
assert fpath.suffix == ".data" and fpath.stem == artifact.uid

In [ ]:
artifact.delete(permanent=True, storage=True)

In [ ]:
path_in_storage.unlink()

In [ ]:
!lamin delete --force {instance_name}